# Creational Pattern Exercise

In [13]:
from abc import ABC, abstractmethod

## Exercise 1: Refactor a Payment System

### 1) The Interface

In [14]:
class PaymentProcessor(ABC):

    @abstractmethod
    def validate(self, details: dict) -> bool:
        pass

    @abstractmethod
    def process(self, amount: float, details: dict) -> dict:
        pass


### 2) Concrete Implementations

In [15]:
class CreditCardProcessor(PaymentProcessor):
    def validate(self, details):
        return (
            details.get("card_number")
            and len(details["card_number"]) == 16
            and details.get("cvv")
            and len(details["cvv"]) == 3
        )

    def process(self, amount, details):
        if not self.validate(details):
            return {"success": False, "error": "Invalid credit card"}

        fee = amount * 0.029
        return {
            "success": True,
            "method": "credit_card",
            "amount": amount + fee,
            "fee": fee
        }

class BankTransferProcessor(PaymentProcessor):
    def validate(self, details):
        return details.get("iban") and len(details["iban"]) >= 15

    def process(self, amount, details):
        if not self.validate(details):
            return {"success": False, "error": "Invalid IBAN"}

        fee = 1.50
        return {
            "success": True,
            "method": "bank_transfer",
            "amount": amount + fee,
            "fee": fee
        }

class PayPalProcessor(PaymentProcessor):
    def validate(self, details):
        return details.get("email") and "@" in details["email"]

    def process(self, amount, details):
        if not self.validate(details):
            return {"success": False, "error": "Invalid PayPal email"}

        fee = amount * 0.034 + 0.30
        return {
            "success": True,
            "method": "paypal",
            "amount": amount + fee,
            "fee": fee
        }

### 3) The Factory

In [16]:
class PaymentFactory:
    _processors = {
        "credit_card": CreditCardProcessor,
        "bank_transfer": BankTransferProcessor,
        "paypal": PayPalProcessor
    }

    def get_processor(self, payment_type: str) -> PaymentProcessor:
        processor_cls = self._processors.get(payment_type)
        if not processor_cls:
            raise ValueError(f"Unknown payment type: {payment_type}")
        return processor_cls()

### Usage:

In [17]:
factory = PaymentFactory()
processor = factory.get_processor("credit_card")
result = processor.process(100, {"card_number":"1234567890123456","cvv":"123"})
print(result)

{'success': True, 'method': 'credit_card', 'amount': 102.9, 'fee': 2.9000000000000004}


## Exercise 2: HR Employee Onboarding System

In [18]:
from dataclasses import dataclass

### 1) The Product

In [19]:
@dataclass
class Employee:
    first_name: str,
    last_name: str,
    email: str,
    department: str,
    position: str,
    salary: float,
    start_date: str,
    manager_id: int = None,
    phone: str = None,
    address: str = None,
    emergency_contact: str = None,
    has_parking: bool = False,
    has_laptop: bool = False,
    has_vpn_access: bool = False,
    has_admin_rights: bool = False,
    office_location: str = None,
    contract_type: str = "permanent"

### 2) The Builder

In [20]:
class EmployeeBuilder:

    def __init__(self):
        self.employee = Employee("", "", "")

    def with_name(self, first, last):
        self.employee.first_name = first
        self.employee.last_name = last
        return self

    def with_email(self, email):
        if "@" not in email:
            raise ValueError("Invalid email")
        self.employee.email = email
        return self

    def with_job(self, dept, position, salary):
        self.employee.department = dept
        self.employee.position = position
        self.employee.salary = salary
        return self

    def with_equipment(self, laptop=False, parking=False):
        self.employee.has_laptop = laptop
        self.employee.has_parking = parking
        return self

    def with_access(self, vpn=False, admin=False):
        self.employee.has_vpn_access = vpn
        self.employee.has_admin_rights = admin
        return self

    def build(self):
        if not self.employee.first_name or not self.employee.last_name:
            raise ValueError("Name required")
        return self.employee

In [24]:
# Expected usage:
# Fluent, readable creation
employee = (
    EmployeeBuilder()
    .with_name("John", "Doe")
    .with_email("john.doe@company.com")
    .with_job("Engineering", "Senior Developer", 75000)
    .with_equipment(laptop=True, parking=False)
    .with_access(vpn=True, admin=True)
    .build()
)

## Exercise 3: Basic Configuration Pattern

In [26]:
import json

In [30]:
class ConfigSource(ABC):
    @abstractmethod
    def load(self):
        pass

class JsonFileConfigSource(ConfigSource):
    def __init__(self, path="config.json"):
        self._path = path

    def load(self):
        with open(self._path, "r", encoding="utf-8") as f:
            return json.load(f)

class ConfigManager:
    _instance = None

    def __init__(self, source):
        self._config = source.load()

    @classmethod
    def get_instance(cls, source=None):
        if cls._instance is None:
            if source is None:
                source = JsonFileConfigSource("config.json")
            cls._instance = cls(source)
        return cls._instance

    def get(self, key, default=None):
        current = self._config
        for part in key.split("."):
            if isinstance(current, dict) and part in current:
                current = current[part]
            else:
                return default
        return current